In [1]:
import json
import os
import warnings

from matplotlib import pyplot as plt
import pandas as pd
import sympy as sp
from sympy.physics.quantum import Dagger

from hamiltonians import *
from qaoa_solver import QAOASolver
from utils import *

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def get_setup(experiment):
    return '_'.join(experiment.split('_')[2:])

experiment_names = os.listdir('experiments')
experiment_configs = []
experiment_results = []

for experiment in experiment_names:
    try:
        with open(f'experiments/{experiment}/{experiment}_conf.json', 'r') as f:
            conf = json.load(f)
            conf['experiment'] = experiment
            conf['setup'] = get_setup(experiment)
            experiment_configs.append(conf)
        
        result = pd.read_json(f'experiments/{experiment}/{experiment}_results.jsonl',lines=True)
        result['experiment'] = experiment
        experiment_results.append(result)
    except ValueError:
        print(f'Error reading experiment {experiment}')

experiment_configs = pd.DataFrame(experiment_configs)
experiment_results = pd.concat(experiment_results)

experiment_results_agg = (
    experiment_results
        .groupby('experiment')
        .agg({
            'iter': 'count',
            'steps': ['mean', 'median', 'std'],
            'cost': ['mean', 'median', 'std'], 
            'fidelity': ['mean', 'median', 'std', 'max', 'min']
        })
    )

experiment_results_agg.columns = ['_'.join(map(str, col)) for col in experiment_results_agg.columns.to_flat_index()]

experiment_results_agg['infidelity_mean'] = 1 - experiment_results_agg['fidelity_mean']
experiment_results_agg['infidelity_median'] = 1 - experiment_results_agg['fidelity_median']
experiment_results_agg['infidelity_min'] = 1 - experiment_results_agg['fidelity_max']
experiment_results_agg['infidelity_max'] = 1 - experiment_results_agg['fidelity_min']

experiments_df = pd.merge(experiment_configs, experiment_results_agg, how='inner', on='experiment')

In [3]:
solver = QAOASolver(N=35, p=4,
                    problem_hamiltonian_gen=default_problem_H,
                    mixer_hamiltonian_gen=default_mixer_H,
                    cost_hamiltonian_gen=abs_cost_H)

In [4]:
solver.nx

2

In [5]:
solver.ny

3

In [6]:
solver.solution

{'01110', '11010'}

In [5]:
experiment = 'N21_p4_Hp=simp_Hc=abs'
df = experiment_results[experiment_results['experiment']==experiment]

In [7]:
df.iloc[95,]

N                                                             21
nx                                                             1
ny                                                             2
iter                                                          95
layers                                                         4
gammas_init                  [2.9, 0.9, 0.9, 0.6000000000000001]
betas_init     [0.7000000000000001, 2.6, 0.1, 0.3000000000000...
gammas         [3.040113209341051, 0.8743817780444371, 0.9778...
betas          [1.539022914605795, 3.891350398663054, 0.90491...
cost                                                    6.048602
steps                                                        923
fidelity                                                0.986683
state          [-0.00834276421912622+0.00810992597439381j, 0....
experiment                                 N21_p4_Hp=simp_Hc=abs
Name: 95, dtype: object

In [9]:
state = df.iloc[2,]['state']

In [10]:
solver = QAOASolver(N=15, p=4,
                    problem_hamiltonian_gen=simplified_problem_H,
                    mixer_hamiltonian_gen=default_mixer_H,
                    cost_hamiltonian_gen=abs_cost_H)

In [11]:
solver.solution

'101'

In [12]:
psi = to_polar(sp.Matrix(state))

In [13]:
Hc = sp.Matrix(solver.Hc.matrix())

In [14]:
psi

Matrix([
[  0.0753842291280339*exp(0.469755684196106*I)],
[    0.103631521812077*exp(2.06015680939675*I)],
[   0.010261989185542*exp(-0.16921289632257*I)],
[   0.0520848975030124*exp(-0.4565578165909*I)],
[0.0980276389387262*exp(-0.0406583291083757*I)],
[   0.171103276823891*exp(-2.44459161732031*I)],
[  0.0125897187191565*exp(0.467124509042339*I)],
[   0.970420182375931*exp(-2.35322116406668*I)]])

In [15]:
Hc

Matrix([
[20.0,   0,    0,   0,   0,   0,   0, 0],
[   0, 8.0,    0,   0,   0,   0,   0, 0],
[   0,   0, 14.0,   0,   0,   0,   0, 0],
[   0,   0,    0, 2.0,   0,   0,   0, 0],
[   0,   0,    0,   0, 6.0,   0,   0, 0],
[   0,   0,    0,   0,   0, 2.0,   0, 0],
[   0,   0,    0,   0,   0,   0, 4.0, 0],
[   0,   0,    0,   0,   0,   0,   0, 0]])

In [16]:
Dagger(psi) * abs(Hc) * psi

Matrix([[0.323314744258535]])

In [20]:
int(solver.solution,2)

5

In [22]:
psi[5]

0.171103276823891*exp(-2.44459161732031*I)

In [23]:
get_population(psi, solver.solution)

0.029276331339873108

In [26]:
df.iloc[95,]['fidelity']

np.float64(0.9866831840987481)